In [1]:
import pandas as pd
import numpy as np
import json
import seaborn as sns
import os
import cv2
from matplotlib import pyplot as plt

In [39]:
#Detection Result Json
data_json_detected = '/opt/ml/detection/dataset/detector_test/result.bbox.json'
#Test json
data_json = '/opt/ml/detection/dataset/val_fold1.json'

In [40]:
with open(data_json, 'r') as outfile:
    data = (json.load(outfile))

df_truth = pd.DataFrame(data['images']) #이미지 데이터
df_truth_annotations = pd.DataFrame(data['annotations'])
df_truth_categories = pd.DataFrame(data['categories'])
df_truth.head(3)

,width,height,file_name,license,flickr_url,coco_url,date_captured,id
0,1024,1024,train/0001.jpg,0,None,None,2021-01-10 16:30:39,1
1,1024,1024,train/0002.jpg,0,None,None,2020-12-27 17:55:52,2
2,1024,1024,train/0013.jpg,0,None,None,2020-12-31 23:57:47,13


In [41]:
with open(data_json_detected, 'r') as outfile:
    data_detected = (json.load(outfile))

df_detected = pd.DataFrame(data_detected) #이미지 데이터
df_detected = pd.DataFrame(df_detected['image_id'].unique(), columns={'id'})
df_detected['detected'] =True

df_tandf = df_truth.merge(df_detected, how='left', on='id')
df_tandf = df_tandf.fillna(False)
df_tandf

,width,height,file_name,license,flickr_url,coco_url,date_captured,id,detected
0,1024,1024,train/0001.jpg,0,False,False,2021-01-10 16:30:39,1,False
1,1024,1024,train/0002.jpg,0,False,False,2020-12-27 17:55:52,2,False
2,1024,1024,train/0013.jpg,0,False,False,2020-12-31 23:57:47,13,True
3,1024,1024,train/0026.jpg,0,False,False,2020-12-30 21:19:54,26,False
4,1024,1024,train/0037.jpg,0,False,False,2021-04-28 23:11:59,37,False
...,...,...,...,...,...,...,...,...,...
984,1024,1024,train/4869.jpg,0,False,False,2020-12-28 10:01:18,4869,False
985,1024,1024,train/4870.jpg,0,False,False,2020-12-19 14:29:18,4870,False
986,1024,1024,train/4873.jpg,0,False,False,2021-01-01 13:08:05,4873,False
987,1024,1024,train/4874.jpg,0,False,False,2020-12-25 13:56:20,4874,False


In [48]:
df_false_detected = df_tandf.loc[df_tandf['detected'] == False]
df_true_detected = df_tandf.loc[df_tandf['detected'] == True]
df_false_detected = df_false_detected.drop('detected', axis=1)
df_true_detected = df_true_detected.drop('detected', axis=1)

d_categories_json = df_truth_categories.to_json(orient='records')
parsed_categories = json.loads(d_categories_json)

final_json_fd = df_false_detected.to_json(orient = 'records')
final_json_fd = json.loads(final_json_fd)
final_json_fd = json.dumps({'images':final_json_fd, 'categories':parsed_categories}, indent=4)
parsed_fd = json.loads(final_json_fd)

final_json_td = df_true_detected.to_json(orient = 'records')
final_json_td = json.loads(final_json_td)
final_json_td = json.dumps({'images':final_json_td, 'categories':parsed_categories}, indent=4)
parsed_td = json.loads(final_json_td)

#경로 매번 수정
file_path_to_model1 = '/opt/ml/detection/dataset/test_result_from_detector/to_model_1.json' 
file_path_to_model2 = '/opt/ml/detection/dataset/test_result_from_detector/to_model_2.json'

with open(file_path_to_model1, 'w') as outfile:
    json.dump(parsed_fd, outfile, indent=4)

with open(file_path_to_model2, 'w') as outfile:
    json.dump(parsed_fd, outfile, indent=4)